# Week 8: QIIME2 "Moving Pictures" tutorial

<font size="3">In this tutorial, we will be analyzing genomic sequence data from the human microbiome. The sample data we will be using is composed of two individuals at four body sites at five timepoints, and the first timepoint was initiated immediately after antibiotic usage. This tutorial is adapted from: https://docs.qiime2.org/2021.8/tutorials/moving-pictures/ </font>  

### Import required packages and data
Let's start by importing our required packages.

In [ ]:
import qiime2 as q2
!mkdir emp-single-end-sequences
!mv barcodes.fastq.gz emp-single-end-sequences/
!mv sequences.fastq.gz emp-single-end-sequences/

### Import data as a QIIME 2 artifact
In order to analyze data with QIIME2, we need to convert it to a QIIME2 artifact. This file structure contains information about the type of data and source of data present.

In [ ]:
!qiime tools import \
    --type EMPSingleEndSequences \
    --input-path emp-single-end-sequences \
    --output-path emp-single-end-sequences.qza

In [ ]:
!qiime tools peek emp-single-end-sequences.qza


### Demultiplexing sequences
QIIME2 artifacts contain sequences that are multiplexed, meaning that they have not yet been assigned to samples. In this step, we will use our sample metadata to demultiplex our reads, and infer which barcodes belong to each sample. 

In [ ]:
##demultiplex reads
!qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza


Let's generate a summary of our demultiplexing step, which will contain information about how many sequences were obtained per sample, and also a summary of the distribution of sequence qualities at each position in our sequence data.

In [ ]:
#summarize
!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

In [ ]:
#Download demux.qzv and drag into https://view.qiime2.org/

*Note: All QIIME2 visualizers will generate a .qzv file, which can be visualized by dragging the file into this webpage: https://view.qiime2.org/

### Sequence quality control and feature table construction: DADA2
DADA2 is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. This pipeline will also filter any phiX reads (commonly present in marker gene Illumina sequence data) as well as chimeric sequences.

- dada2 denoise-single parameters:
    - --p-trim-left: trims off the first m bases of each sequence
    - --p-trunc-len: truncates each sequence at position n

**Values for these parameters can be determined by reviewing the Interactive Quality Plot tab in the demux.qzv file generated above*

### Questions: 
What is the median number of reads per sample?

Based on the plots you see in demux.qzv, what values would you choose for --p-trunc-len and --p-trim-left in this case? Please input below.

In [ ]:
##DADA2
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left  \
  --p-trunc-len  \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --o-denoising-stats stats-dada2.qza

Convert output statistics to a visualization file in order to view output

In [ ]:
!qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

### Generate visual summaries of our data after filtering so far
- <ins>feature-table summarize</ins>: gives information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics
- <ins>feature-table tabulate-seqs</ins>: provides a mapping of feature IDs to sequences, and provides links to easily BLAST each sequence against the NCBI nt database

In [ ]:
#feature table
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

### Questions
Which sample has the lowest feature count?
How many features / ASVs were identified?
Which body site has samples with the lowest coverage?

### Generate a tree for phylogenetic diversity analyses
QIIME2 can generate several phylogenetic diversity metrics, but these metrics require a rooted phylogenetic tree relating features of samples to each other. This tree can be generated with the phylogeny <ins>align-to-tree-mafft-fasttree</ins> command.

First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [ ]:
#Phylogenetic tree for diversity analyses
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

### Question:
View the table.qzv QIIME 2 artifact, and in particular the Interactive Sample Detail tab in that visualization. What value would you choose to pass for --p-sampling-depth? How many samples will be excluded from your analysis based on this choice? How many total sequences will you be analyzing in the core-metrics-phylogenetic command?

### Alpha and Beta diversity analysis
The diversity core-metrics-phylogenetic plugin computes various alpha and beta diversity metrics and generates principle coordinates analysis (PCoA) plots for each of the beta diversity metrics.

- --p-sampling-depth parameter: the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter.

Alpha diversity metrics:
- Shannon’s diversity index (a quantitative measure of community richness)
- Observed Features (a qualitative measure of community richness)
- Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)
- Evenness (or Pielou’s Evenness; a measure of community evenness)

Beta diversity metrics:
- Jaccard distance (a qualitative measure of community dissimilarity)
- Bray-Curtis distance (a quantitative measure of community dissimilarity)
- unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
- weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth  \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

### Questions:
Which metric separates the samples best by body site?
How does unweighted vs. weighted unifrac compare? What does this mean?

Now let's test for associations between categorical metadata columns and alpha diversity data. We’ll do that here for the Faith Phylogenetic Diversity (a measure of community richness) and evenness metrics.

In [ ]:
#Calculate significance
#Alpha
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv


In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

### Questions:
Which body site is the most diverse? Least diverse?
Most even? Least even?
In this dataset does antibiotics significantly change diversity?


### Taxonomic analysis
In this section we will explore the taxonomic composition of our samples. 

To begin we will assign taxonomy to our sequences using a pre-trained Naive Bayes classifier and the q2-feature-classifier plugin. Once we apply this classifier to our sequences, we can generate a visualization of the resulting mapping from sequence to taxonomy.

In [ ]:
#Taxonomic analysis
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza


In [ ]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv


Next, we can view the taxonomic composition of our samples with interactive bar plots generated by the following command. We can view these plots using the QIIME2 visulization webtool (https://view.qiime2.org/)

In [ ]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

### Differential abundance testing with ANCOM
ANCOM can be applied to identify features that are present in different abundances across sample groups. Because we expect a lot of features to change in abundance across body sites, in this tutorial we’ll filter our full feature table to only contain gut samples. We’ll then apply ANCOM to determine which, if any, sequence variants and genera are differentially abundant across the gut samples of our two subjects. 

The command below will filter out only gut samples, and create a feature table to use in future steps.

In [ ]:
#Differential abundance testing with ANCOM
#Filter table
!qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "[body-site]='gut'" \
  --o-filtered-table gut-table.qza

Since ANCOM cannot tolerate frequencies of features on a per-sample basis being zero, we will use the add-pseudocount method to modify our QIIME2 artifact.

In [ ]:
#add zero
!qiime composition add-pseudocount \
  --i-table gut-table.qza \
  --o-composition-table comp-gut-table.qza

Now let's run ANCOM on the subject column to determine what features differ in abundance across the gut samples of the two subjects.

In [ ]:
#Differ in abundance across 2 subjects
!qiime composition ancom \
  --i-table comp-gut-table.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization ancom-subject.qzv


We’re also often interested in performing a differential abundance test at a specific taxonomic level. To do this, we can collapse the features in our FeatureTable at the taxonomic level of interest, and then re-run the above steps. In this tutorial, we collapse our feature table at the genus level (i.e. level 6 of the Greengenes taxonomy).

In [ ]:
#SWitch to genus level
!qiime taxa collapse \
  --i-table gut-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table gut-table-l6.qza


In [ ]:
!qiime composition add-pseudocount \
  --i-table gut-table-l6.qza \
  --o-composition-table comp-gut-table-l6.qza


In [ ]:
!qiime composition ancom \
  --i-table comp-gut-table-l6.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization l6-ancom-subject.qzv